In [1]:
import os

assert os.environ['CONDA_DEFAULT_ENV']=='ttv'

## TOIs

In [2]:
import sys
sys.path.insert(0, '../code')
from utils import *

In [3]:
df = get_tois(clobber=True)
df.tail()

Loaded:  ../data/TOIs.csv


,TIC ID,TOI,Previous CTOI,Master,SG1A,SG1B,SG2,SG3,SG4,SG5,...,Stellar Radius (R_Sun) err,Stellar Metallicity,Stellar Metallicity err,Stellar Mass (M_Sun),Stellar Mass (M_Sun) err,Sectors,Date TOI Alerted (UTC),Date TOI Updated (UTC),Date Modified,Comments
6088,400466395,6952.01,NaN,3,4,3,3,1,4,4,...,0.06,-0.639035,0.009111,1.097,0.146502,"43,44,71,72",2024-02-29,2024-02-29,2024-03-03 12:12:56,found in faint-star QLP search
6089,70435923,6953.01,NaN,3,4,3,3,1,4,4,...,0.14,0.087000,0.214000,1.090,0.162627,"44,45,46,71,72",2024-02-29,2024-02-29,2024-03-03 12:12:56,found in faint-star QLP search
6090,443607377,6954.01,NaN,2,4,2,2,1,4,4,...,0.06,0.030000,0.100000,0.834,0.098055,"9,36,45,46,72",2024-02-29,2024-02-29,2024-03-03 12:12:56,found in faint-star QLP search
6091,81963214,6955.01,NaN,3,4,3,3,1,4,4,...,0.14,0.079000,0.014000,1.350,0.218986,"43,44,45,71,72",2024-02-29,2024-02-29,2024-03-03 12:12:56,found in faint-star QLP search
6092,294350792,6956.01,NaN,3,4,3,3,1,4,4,...,NaN,-0.096000,0.038000,1.470,NaN,"33,43,44,45,71,72",2024-02-29,2024-02-29,2024-03-03 12:12:56,found in faint-star QLP search


In [4]:
tics = df['TIC ID'].unique()
len(tics)

5841

In [5]:
len(df['TOI'].unique())

6093

In [6]:
nsys = df['TIC ID'].value_counts()
nsys

TIC ID
425997655    5
367630162    5
29781292     4
251848941    4
142276270    4
            ..
47587394     1
419148828    1
172630205    1
437156845    1
294350792    1
Name: count, Length: 5841, dtype: int64

In [7]:
tics_multis = list(nsys[nsys>1].index)
print(len(tics_multis))

198


## TOI 1136 (Dai+2023)
First order MMR: 3:2, 2:1, 3:2, 7:5, and 3:2

In [8]:
col = 'Period (days)'
d = df[df['TIC ID']==142276270].sort_values(by=col)
d[col]

825     6.258792
824    12.518728
827    18.804756
826    26.317954
Name: Period (days), dtype: float64

In [9]:
# add two outer planets
periods = [4.17278, 6.25725, 12.51937, 18.7992, 26.3162, 39.5387]
periods

[4.17278, 6.25725, 12.51937, 18.7992, 26.3162, 39.5387]

https://arxiv.org/abs/1208.3312

$$
\frac{p'}{p} \sim \frac{j}{j-1}
$$

proximity to resonance:
$$
\Delta = \frac{p'}{p} \frac{j-1}{j} - 1
$$

In [10]:
get_orbit_pairs(10, order=1)

[(1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9)]

In [11]:
resonances_1st_order, deltas = get_resonant_pairs(periods, order=1, tol=0.01)
resonances_1st_order, deltas

(['3:2 | P=(4.17,6.26) (delta=0.03%)',
  '2:1 | P=(6.26,12.52) (delta=0.04%)',
  '3:2 | P=(12.52,18.80) (delta=0.11%)',
  '3:2 | P=(26.32,39.54) (delta=0.16%)'],
 [0.0003067499364933779,
  0.0003891485876383882,
  0.001072737685682057,
  0.0016314412161837488])

In [12]:
resonances_2nd_order, deltas = get_resonant_pairs(periods, order=2, tol=0.01)
resonances_2nd_order, deltas

(['6:4 | P=(4.17,6.26) (delta=0.03%)',
  '4:2 | P=(6.26,12.52) (delta=0.04%)',
  '6:4 | P=(12.52,18.80) (delta=0.11%)',
  '7:5 | P=(18.80,26.32) (delta=0.01%)',
  '6:4 | P=(26.32,39.54) (delta=0.16%)'],
 [0.0003067499364933779,
  0.0003891485876383882,
  0.001072737685682057,
  0.00010182804131475542,
  0.0016314412161837488])

## All TOIs

In [27]:
ttv_1st_order = []       # commensurability in 1 planet pair 
ttv_2nd_order = []
ttv_chain = []           # more than 1 planet pair
deltas1 = []
deltas2 = []
texts = []
for tic in tics_multis:
    dd = df[df['TIC ID']==tic].squeeze()
    periods = dd["Period (days)"].values
    Nplanets = len(periods)
    toi = str(dd.iloc[0].TOI).split('.')[0] 
    
    resonances_1st_order, delta1 = get_resonant_pairs(periods, order=1, tol=0.01);
    resonances_2nd_order, delta2 = get_resonant_pairs(periods, order=2, tol=0.01);
    msg = f"TOI-{str(toi).zfill(4)} | [TIC-{tic}](https://exofop.ipac.caltech.edu/tess/target.php?id={tic}) (nplanets={Nplanets}):<br>\n"
    show = False
    if len(resonances_1st_order)>=1:
        msg += f"First Order: {' '.join(resonances_1st_order)}<br>\n"
        ttv_1st_order.append(tic)
        deltas1.append(min(delta1))
        if len(resonances_1st_order)>=2:
            ttv_chain.append(tic)         
        show = True
    if len(resonances_2nd_order)>=1:
        if len(resonances_1st_order)>=1:
            i = int(resonances_1st_order[0].split(':')[0])
            j = int(resonances_2nd_order[0].split(':')[0])
            if j/i!=2:
                msg += f"Second Order: {' '.join(resonances_2nd_order)}\n"
                ttv_2nd_order.append(tic)
                deltas2.append(min(delta2))
                show = True
                
        else:
            msg += f"Second Order: {' '.join(resonances_2nd_order)}\n"
            ttv_2nd_order.append(tic)
            deltas2.append(min(delta2))
            show = True
    if show:
        print(msg)
        texts.append(msg)

TOI-0174 | [TIC-425997655](https://exofop.ipac.caltech.edu/tess/target.php?id=425997655) (nplanets=5):<br>
First Order: 2:1 | P=(3.98,7.91) (delta=0.58%)<br>

TOI-2104 | [TIC-367630162](https://exofop.ipac.caltech.edu/tess/target.php?id=367630162) (nplanets=5):<br>
Second Order: 5:3 | P=(1.98,3.27) (delta=0.85%)

TOI-0282 | [TIC-29781292](https://exofop.ipac.caltech.edu/tess/target.php?id=29781292) (nplanets=4):<br>
First Order: 3:2 | P=(56.00,84.26) (delta=0.30%)<br>

TOI-1136 | [TIC-142276270](https://exofop.ipac.caltech.edu/tess/target.php?id=142276270) (nplanets=4):<br>
First Order: 2:1 | P=(6.26,12.52) (delta=0.01%) 3:2 | P=(12.52,18.80) (delta=0.14%)<br>

TOI-0700 | [TIC-150428135](https://exofop.ipac.caltech.edu/tess/target.php?id=150428135) (nplanets=4):<br>
First Order: 4:3 | P=(27.81,37.42) (delta=0.93%)<br>

TOI-2267 | [TIC-459837008](https://exofop.ipac.caltech.edu/tess/target.php?id=459837008) (nplanets=3):<br>
First Order: 9:8 | P=(2.03,2.29) (delta=0.02%)<br>

TOI-0175 |

/home/jp/github/research/project/young_ttvs/notebooks/../code/utils.py:302: RuntimeWarning: divide by zero encountered in double_scalars
  delta = abs((Pout/Pin)*(i/j) - 1)


In [28]:
len(ttv_1st_order), len(ttv_2nd_order)

(22, 9)

In [29]:
import pandas as pd

s1 = pd.DataFrame(np.c_[ttv_1st_order,deltas1], columns=['tic','delta_min'])
s1['1st order'] = True
s1.sort_values(by='delta_min')

,tic,delta_min,1st order
7,441798995.0,0.000023,True
2,142276270.0,0.000091,True
4,459837008.0,0.000158,True
21,384984325.0,0.000488,True
16,455947620.0,0.001917,True
17,143022742.0,0.002051,True
15,303204165.0,0.002446,True
1,29781292.0,0.003024,True
13,153065527.0,0.004089,True
10,259377017.0,0.005161,True


In [30]:
s2 = pd.DataFrame(np.c_[ttv_2nd_order,deltas2], columns=['tic','delta_min'])
s2['2nd order'] = True
s2.sort_values(by='delta_min')

,tic,delta_min,2nd order
4,178155732.0,0.000263,True
8,294471966.0,0.001432,True
2,27491137.0,0.002862,True
1,441739020.0,0.006611,True
5,468979441.0,0.006748,True
3,397362481.0,0.007041,True
6,392681545.0,0.007379,True
0,367630162.0,0.008545,True
7,344927749.0,0.009826,True


In [31]:
res = pd.merge(s1, s2, how='outer')
res = res.replace(np.nan, False)
res['tic'] = res.tic.map(int)
res = res.sort_values(by='delta_min')
res.head()

,tic,delta_min,1st order,2nd order
7,441798995,0.000023,True,False
2,142276270,0.000091,True,False
4,459837008,0.000158,True,False
26,178155732,0.000263,False,True
21,384984325,0.000488,True,False


In [32]:
len(res)

31

In [33]:
len(texts), len(res)

(31, 31)

In [34]:
for tic in res.tic:
    idx = [True if i.split('TIC-')[1].split(']')[0]==str(tic) else False for i in texts]
    x = np.array(texts)[idx]
    print(x[0] if len(x)>0 else '')

TOI-2269 | [TIC-441798995](https://exofop.ipac.caltech.edu/tess/target.php?id=441798995) (nplanets=3):<br>
First Order: 2:1 | P=(1.42,2.84) (delta=0.00%)<br>

TOI-1136 | [TIC-142276270](https://exofop.ipac.caltech.edu/tess/target.php?id=142276270) (nplanets=4):<br>
First Order: 2:1 | P=(6.26,12.52) (delta=0.01%) 3:2 | P=(12.52,18.80) (delta=0.14%)<br>

TOI-2267 | [TIC-459837008](https://exofop.ipac.caltech.edu/tess/target.php?id=459837008) (nplanets=3):<br>
First Order: 9:8 | P=(2.03,2.29) (delta=0.02%)<br>

TOI-0396 | [TIC-178155732](https://exofop.ipac.caltech.edu/tess/target.php?id=178155732) (nplanets=3):<br>
Second Order: 5:3 | P=(3.59,5.97) (delta=0.03%)

TOI-6109 | [TIC-384984325](https://exofop.ipac.caltech.edu/tess/target.php?id=384984325) (nplanets=2):<br>
First Order: 3:2 | P=(5.70,8.54) (delta=0.05%)<br>

TOI-1446 | [TIC-294471966](https://exofop.ipac.caltech.edu/tess/target.php?id=294471966) (nplanets=2):<br>
Second Order: 7:5 | P=(4.51,6.32) (delta=0.14%)

TOI-6022 | [TIC

In [ ]:
res.to_csv('../data/tics_resonance_from_toi.csv')

In [52]:
idx = df['TIC ID'].isin(res.tic)
tois_in_resonance = df['TOI'][idx].apply(lambda x: str(x).split('.')[0].zfill(4)).unique()
tois_in_resonance

array(['0174', '0175', '0216', '0270', '0282', '0396', '0406', '0663',
       '0696', '0700', '1097', '1136', '1339', '1446', '1670', '1730',
       '1749', '2076', '2104', '2267', '2269', '4323', '4495', '4647',
       '5493', '5520', '6022', '6054', '6109', '6265', '6276'],
      dtype=object)

In [54]:
np.savetxt('../data/tics_resonance_from_toi.txt', res.tic.tolist(), fmt="%d")
np.savetxt('../data/tois_resonance_from_toi.txt', tois_in_resonance, fmt="%s")
np.savetxt('../data/tics_resonance_from_toi_1st_order.txt', ttv_1st_order, fmt="%d")
np.savetxt('../data/tics_resonance_from_toi_2nd_order.txt', ttv_2nd_order, fmt="%d")

## Transit timing database
* https://ui.adsabs.harvard.edu/abs/2022ApJS..259...62I/abstract

In [26]:
#https://github.com/transit-timing/tt/blob/master/3_database/table4.csv
url = "https://raw.githubusercontent.com/transit-timing/tt/master/3_database/table4.csv"
df2 = pd.read_csv(url)
df2

,System,Orbit number,T_mid,Uncertainty (days),Time System,#,Reference
0,CoRoT-01,-1412,2.454138e+06,0.000470,BJD,1,2009A&A...506..369B
1,CoRoT-01,-1411,2.454140e+06,0.000590,BJD,1,2009A&A...506..369B
2,CoRoT-01,-1410,2.454141e+06,0.000620,BJD,1,2009A&A...506..369B
3,CoRoT-01,-1409,2.454143e+06,0.000390,BJD,1,2009A&A...506..369B
4,CoRoT-01,-1408,2.454144e+06,0.001630,BJD,1,2009A&A...506..369B
...,...,...,...,...,...,...,...
8662,XO-7,214,2.459407e+06,0.000510,BJD_TDB,1,This work
8663,XO-7,215,2.459410e+06,0.000467,BJD_TDB,1,This work
8664,XO-7,216,2.459413e+06,0.000508,BJD_TDB,1,This work
8665,XO-7,217,2.459415e+06,0.000517,BJD_TDB,1,This work
